In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
from knockouts import *

In [2]:
MeanTCounts = pandas.read_csv("./ibdmdb/metaT_per_diagnosis.csv",index_col=0)
NormalizedTCounts = MeanTCounts.div(MeanTCounts.sum())
NormalizedTCounts.head()

,nonIBD,UC,CD,IBD
#FeatureID,,,,
HMPREF9436_RS14285,0.078333,0.039315,0.037492,0.037492
HMPREF9436_RS14685,0.023592,0.008788,0.013451,0.013451
BILO145876EF_RS04385,0.022452,0.003468,0.005717,0.005717
FAEPRAM212_RS14865,0.017201,0.023327,0.030562,0.030562
BSFG_RS23450,0.010633,0.011724,0.010224,0.010224


In [3]:
# condition = "nonIBD"
condition = "IBD"
# condition = "UC"
# condition = "CD"

In [4]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/reduction/reduction_{}.log".format(condition), filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [5]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
done = set(i.split("_killable")[0] for i in os.listdir("./ibdmdb/reduction/{}".format(condition)) if "txt" in i)
run_for = survivors - done
len(run_for)

7

In [6]:
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    kill_genes = get_targets(model,condition,NormalizedTCounts)
    index_dct = {r.id : model.reactions.index(r) for r in model.reactions}
    
    pandas.DataFrame(columns=kill_genes).T.to_csv("./ibdmdb/reduction_targets/{}/{}_targets.txt".format(condition,org),header=None)
    
    me_nlp = get_nlp(model)
    del model
    
    f,basis = get_feasibility(me_nlp)
    killable = get_killable(index_dct,me_nlp,basis,kill_genes,limit=lambda x:2.1917479162633475e-13,ListHandler=ListHandler,org=org)
    
    pandas.DataFrame(columns=killable).T.to_csv("./ibdmdb/reduction/{}/{}_killable.txt".format(condition,org),header=None)

In [ ]:
NP = min([14,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
    
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (7 threads):   0%|          | 0/7 [00:00<?, ?it/s]

Loading Providencia_rustigianii_DSM_4541Loading Morganella_morganii_subsp_morganii_KTLoading Yersinia_kristensenii_ATCC_33638Loading Gordonia_rubripertincta_NBRC_101908Loading Escherichia_fergusonii_ATCC_35469Loading Vibrio_mimicus_MB_451Loading Lysinibacillus_fusiformis_ZB2






Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp9u5d6688.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpihp2azv9.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpplbolszu.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model 